<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [ ]:

from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [5]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [6]:
data_generator = ImageDataGenerator(
  preprocessing_function=preprocess_input
)

In [4]:
train_generator = data_generator.flow_from_directory(
  'concrete_data_week4/concrete_data_week4/train',
  target_size=(image_resize, image_resize),
  batch_size=batch_size_training,
  class_mode='categorical'
)

Found 30001 images belonging to 2 classes.


In [5]:
validation_generator = data_generator.flow_from_directory(
  'concrete_data_week4/concrete_data_week4/valid',
  target_size=(image_resize, image_resize),
  batch_size=batch_size_validation,
  class_mode='categorical'
)

Found 9501 images belonging to 2 classes.


In [6]:
vgg16Model = Sequential()

In [7]:
vgg16Model.add(VGG16(
  include_top=False,
  pooling='avg',
  weights='imagenet',
))

In [8]:
vgg16Model.add(Dense(num_classes, activation='softmax'))

In [9]:
vgg16Model.layers

[<Functional name=vgg16, built=True>, <Dense name=dense, built=True>]

In [10]:
vgg16Model.layers[0]

<Functional name=vgg16, built=True>

In [11]:
vgg16Model.layers[0].trainable = False

In [12]:
vgg16Model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 512)            │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,714 (56.14 MB)

 Trainable params: 1,026 (4.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [13]:
vgg16Model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.CategoricalCrossentropy(),
)

In [14]:
# steps_per_epoch_training = len(train_generator)
# steps_per_epoch_validation = len(validation_generator)
steps_per_epoch_training = 30 # reduce data due to memory limitation
steps_per_epoch_validation = 10 # reduce data due to memory limitation
num_epochs = 1

vgg16Model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

c:\Users\ngoch\AppData\Local\pypoetry\Cache\virtualenvs\ibm-ai-engineering-professional-n6kARCg4-py3.12\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


30/30 ━━━━━━━━━━━━━━━━━━━━ 948s 32s/step - loss: 0.3967 - val_loss: 0.1164


In [15]:
vgg16Model.save('vgg16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [35]:
resnet50Model = load_model('../Module 03/classifier_resnet_model.h5')

In [7]:
test_generator = data_generator.flow_from_directory(
  'concrete_data_week4/concrete_data_week4/test',
  target_size=(image_resize, image_resize),
  shuffle=False,
  class_mode='categorical'
)

Found 500 images belonging to 2 classes.


In [18]:
results = vgg16Model.evaluate(test_generator)
print("test loss, test acc:", results)

c:\Users\ngoch\AppData\Local\pypoetry\Cache\virtualenvs\ibm-ai-engineering-professional-n6kARCg4-py3.12\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 145s 9s/step - loss: 0.1194
test loss, test acc: 0.11091490089893341


In [19]:
results = resnet50Model.evaluate(test_generator)
print("test loss, test acc:", results)

16/16 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.9990 - loss: 0.0124
test loss, test acc: [0.012531929649412632, 0.9980000257492065]


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [ ]:
vgg16Model = load_model('vgg16_model.h5')

In [48]:
small_test_data = test_generator[0][0][:10]

In [49]:
vgg16_predictions = vgg16Model.predict(small_test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [50]:
for idx, prediction in enumerate(vgg16_predictions):
  if prediction.argmax() == 1:
    print('Positive')
  else:
    print('Negative')

  # plt.imshow(test_generator[0][0][idx].astype(np.uint8))
  # plt.show()

Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative


In [51]:
resnet50_predictions = resnet50Model.predict(small_test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [52]:
for idx, prediction in enumerate(resnet50_predictions):
  if prediction.argmax() == 1:
    print('Positive')
  else:
    print('Negative')

  # plt.imshow(test_generator[0][0][idx].astype(np.uint8))
  # plt.show()

Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).